In [11]:
import os
import re
import numpy as np
import base64
import requests
from tqdm import tqdm
from PIL import Image
import fitz
import io
import pandas as pd
import random
import ast
from openai import OpenAI
import json
import fitz
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

In [12]:
from dotenv import load_dotenv 
load_dotenv()

True

In [13]:
client = OpenAI()

In [ ]:
# 이미지를 base64 형식으로 인코딩하는 함수
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 이미지가 다단인지 확인하는 함수
def is_two_column(img_array, threshold=240, column_width=0.1):
    height, width = img_array.shape[:2]
    center_start = int(width * (0.5 - column_width/2))
    center_end = int(width * (0.5 + column_width/2))

    center_region = img_array[:, center_start:center_end]

    # 이미지가 컬러라면 RGB를 그레이스케일로 변환
    if len(img_array.shape) == 3:
        center_region = np.mean(center_region, axis=2)

    vertical_projection = np.mean(center_region, axis=1)
    white_ratio = np.mean(vertical_projection > threshold)

    return white_ratio > 0.9  # 중심 부분이 90% 이상 흰색일 때 다단으로 간주
# PNG 이미지를 최적화하는 함수
def optimize_png(img):
    img_buffer = io.BytesIO()
    img.save(img_buffer, format='PNG', optimize=True, compress_level=6)
    optimized_img = Image.open(img_buffer)
    return optimized_img

# PDF 파일을 처리하여 각 페이지를 이미지로 저장하는 함수
def process_pdf(pdf_path, output_folder, dpi=300):
    doc = fitz.open(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    for page_num in range(len(doc)):
        page = doc[page_num]
        zoom = dpi / 72
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat, alpha=False)

        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        img_array = np.array(img)

        # 이미지 최적화 후 저장
        optimize_png(img).save(os.path.join(output_folder, f'page_{page_num+1}.png'))

    doc.close()

In [27]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))
print(os.getcwd())

/Users/seobi/PythonProjects/AxDeepScholar/src/core/rag


In [28]:
def get_project_root(marker="pyproject.toml"): # 현재 프로젝트의 경로를 가져오기 위해 사용용
    cur = Path(__file__).resolve() if "__file__" in globals() else Path().resolve()
    return next((p for p in [cur, *cur.parents] if (p / marker).exists()), cur)    

In [33]:
print(get_project_root())
print(type(get_project_root()))

/Users/seobi/PythonProjects/AxDeepScholar
<class 'pathlib.PosixPath'>


In [ ]:
import os

def get_all_pdfs():
    target_dirs = [str(get_project_root() /"src"/ "data" / "agent"), str(get_project_root() /"src"/ "data"/"rag")]
    pdf_paths = []
    for root_dir in target_dirs:
        for root, _, files in os.walk(root_dir):
            
            for f in files:
                if f.lower().endswith(".pdf"):
                    pdf_paths.append(os.path.join(root, f))

    return sorted(pdf_paths)

In [35]:
get_all_pdfs()

/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent


['/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.00114v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.04788v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.15594v5.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.15692v2.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2501.00083v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2502.14143v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2503.10970v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2504.03646459v1.full.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2504.16736v3.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/a

In [48]:
import os

pdf_files = get_all_pdfs()
for pdf_path in pdf_files:
    print("PDF 처리 중:", pdf_path)

    # PDF 파일명에서 .pdf 제거 후 폴더명 만들기
    output_folder = pdf_path.replace(".pdf", "_pages")

    # 폴더 생성
    os.makedirs(output_folder, exist_ok=True)

    # 이미지 캡쳐 실행
    process_pdf(pdf_path, output_folder)

    print("완료:", output_folder)
# pdf_files

/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent


['/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.00114v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.04788v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.15594v5.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.15692v2.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2501.00083v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2502.14143v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2503.10970v1.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2504.03646459v1.full.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2504.16736v3.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full.pdf',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/a

In [ ]:
def has_meaningful_content(text):
    """텍스트가 의미있는 내용을 포함하는지 확인"""
    if not text:
        return False

    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    # 특수문자 디코딩
    text = text.replace('&lt;', '<').replace('&gt;', '>').replace('&amp;', '&')
    # 공백, 특수문자 제거
    text = re.sub(r'[\s\t\n\r\f\v]+', '', text)
    # div, page 같은 일반적인 태그 관련 단어 제거
    text = re.sub(r'(div|page|style|width|height|pt)', '', text, flags=re.IGNORECASE)

    # 숫자와 단위 제거
    text = re.sub(r'\d+\.?\d*\s*(pt|px|em|rem|%)', '', text)

    # 남은 텍스트가 있는지 확인
    return bool(text.strip())

def extract_content_with_fallback(page):
    """여러 방법을 순차적으로 시도하여 의미있는 내용 추출"""
    content = None
    error_messages = []

    # 1. 기본 텍스트 추출 시도
    try:
        text = page.get_text("text")
        if text and has_meaningful_content(text):
            root = ET.Element("page")
            text_elem = ET.SubElement(root, "text")
            text_elem.text = text.strip()
            return ET.tostring(root, encoding='utf-8').decode('utf-8'), None
    except Exception as e:
        error_messages.append(f"기본 텍스트 추출 실패: {str(e)}")

    # 2. 블록 단위 추출 시도
    try:
        blocks = page.get_text("blocks")
        meaningful_blocks = []
        for block in blocks:
            if has_meaningful_content(block[4]):
                meaningful_blocks.append(block[4].strip())

        if meaningful_blocks:
            root = ET.Element("page")
            blocks_elem = ET.SubElement(root, "blocks")
            for block_text in meaningful_blocks:
                block_elem = ET.SubElement(blocks_elem, "block")
                block_elem.text = block_text
            return ET.tostring(root, encoding='utf-8').decode('utf-8'), None
    except Exception as e:
        error_messages.append(f"블록 추출 실패: {str(e)}")

    # 3. 단어 단위 추출 시도
    try:
        words = page.get_text("words")
        meaningful_words = []
        for word in words:
            if has_meaningful_content(word[4]):
                meaningful_words.append(word[4].strip())

        if meaningful_words:
            root = ET.Element("page")
            words_elem = ET.SubElement(root, "words")
            for word_text in meaningful_words:
                word_elem = ET.SubElement(words_elem, "word")
                word_elem.text = word_text
            return ET.tostring(root, encoding='utf-8').decode('utf-8'), None
    except Exception as e:
        error_messages.append(f"단어 추출 실패: {str(e)}")

    # 4. rawdict 시도
    try:
        raw_dict = page.get_text("rawdict")
        if "blocks" in raw_dict:
            meaningful_text = []
            for block in raw_dict["blocks"]:
                if "lines" in block:
                    for line in block["lines"]:
                        if "spans" in line:
                            for span in line["spans"]:
                                if has_meaningful_content(span.get("text", "")):
                                    meaningful_text.append(span["text"].strip())

            if meaningful_text:
                root = ET.Element("page")
                text_elem = ET.SubElement(root, "text")
                text_elem.text = " ".join(meaningful_text)
                return ET.tostring(root, encoding='utf-8').decode('utf-8'), None
    except Exception as e:
        error_messages.append(f"rawdict 추출 실패: {str(e)}")

    # 모든 방법이 실패했거나 의미있는 내용이 없는 경우 None 반환
    return None, "\n".join(error_messages)

def convert_pdf_to_xml(pdf_path, output_folder):
    """PDF를 XML로 변환하는 메인 함수"""
    if not os.path.exists(pdf_path):
        print(f"PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return

    os.makedirs(output_folder, exist_ok=True)

    print(f"PDF 파일 열기: {pdf_path}")
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        print(f"페이지 {page_num + 1}/{len(doc)} 처리 중...")
        page = doc[page_num]

        content, errors = extract_content_with_fallback(page)
        output_file = os.path.join(output_folder, f"page_{page_num+1:02d}.xml")

        if content:
            print(f"XML 파일 저장 중: {output_file}")
            with open(output_file, "w", encoding="utf-8") as f:
                f.write(content)
            print(f"페이지 {page_num + 1} 처리 완료")
        else:
            print(f"페이지 {page_num + 1}에서 추출된 의미있는 내용이 없습니다.")
            # 빈 내용일 경우 파일 생성하지 않음
            if os.path.exists(output_file):
                os.remove(output_file)

    doc.close()
    print("모든 페이지 처리 완료")

['/Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/img',
 '/Users/seobi/PythonProjects/AxDeepScholar/src/data/rag/img']

In [49]:
import os
for pdf_path in pdf_files:
    print("PDF 처리 중:", pdf_path)
    # PDF 파일명에서 .pdf 제거 후 폴더명 만들기
        # PDF 파일명에서 .pdf 제거 후 폴더명 만들기
    output_folder = pdf_path.replace(".pdf", "_pages")
    # 폴더 생성
    os.makedirs(output_folder, exist_ok=True)
    # 이미지 캡쳐 실행
    convert_pdf_to_xml(pdf_path, output_folder)
    print("완료:", output_folder)

PDF 처리 중: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1.pdf
PDF 파일 열기: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1.pdf
페이지 1/22 처리 중...
XML 파일 저장 중: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1_pages/page_01.xml
페이지 1 처리 완료
페이지 2/22 처리 중...
XML 파일 저장 중: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1_pages/page_02.xml
페이지 2 처리 완료
페이지 3/22 처리 중...
XML 파일 저장 중: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1_pages/page_03.xml
페이지 3 처리 완료
페이지 4/22 처리 중...
XML 파일 저장 중: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1_pages/page_04.xml
페이지 4 처리 완료
페이지 5/22 처리 중...
XML 파일 저장 중: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1_pages/page_05.xml
페이지 5 처리 완료
페이지 6/22 처리 중...
XML 파일 저장 중: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2408.07199v1_pages/page_06.xml
페이지 6 처리 완료
페이지

In [56]:
# 디렉토리에서 매칭되는 XML과 PNG 파일들을 가져오는 함수
def get_matched_files(directory):
    files = os.listdir(directory)

    xml_files = {}
    png_files = {}

    # XML과 PNG 파일들을 분류
    for file in files:
        match = re.match(r'page_(\d+)\.(xml|png)$', file)
        if match:
            number = int(match.group(1))
            extension = match.group(2)
            if extension == 'xml':
                xml_files[number] = file
            elif extension == 'png':
                png_files[number] = file

    # 모든 페이지 번호를 결합하여 매칭
    all_numbers = set(xml_files.keys()).union(set(png_files.keys()))

    matched_files = []
    for number in sorted(all_numbers):
        xml_path = os.path.join(directory, xml_files[number]) if number in xml_files else 'no xml'
        png_path = os.path.join(directory, png_files[number]) if number in png_files else 'no png'
        matched_files.append((xml_path, png_path))

    return matched_files


def get_all_page_folders(root_dir):
    page_dirs = []
    for entry in os.listdir(root_dir):
        full_path = os.path.join(root_dir, entry)
        if os.path.isdir(full_path) and entry.endswith("_pages"):
            page_dirs.append(full_path)
    return page_dirs


def collect_all_matched_files(root_dir):
    page_dirs = get_all_page_folders(root_dir)

    result = {}

    for page_dir in page_dirs:
        matched = get_matched_files(page_dir)
        result[page_dir] = matched  # 폴더별로 저장

    return result

In [62]:
root = str(get_project_root() /"src"/ "data" / "agent")
all_data = collect_all_matched_files(root)

for folder, matched_files in all_data.items():
    print("📁 FOLDER:", folder)
    for xml, png in matched_files:
        print("  -", xml, png)


📁 FOLDER: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages
  - /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/page_01.xml /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/page_1.png
  - /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/page_02.xml /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/page_2.png
  - /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/page_03.xml /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/page_3.png
  - /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/page_04.xml /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/page_4.png
  - /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.

In [64]:
type(all_data)

dict

In [ ]:
all_data.update(collect_all_matched_files(str(get_project_root() /"src"/ "data" / "rag")))

In [68]:
# XML이 있는 경우 사용할 시스템 프롬프트
# system_prompt = '''당신이 해석할 이미지는 보험 약관입니다.
# 1. 중요한 내용이므로 요약하지말고 문법에 신경쓰면서 보이는 그대로 작성해주세요.
# 2. 내용을 임의로 바꾸지 마세요. 그리고 보이는 모든 내용을 다 적으십시오.
# 3. 단, 테이블은 풀어서 평문 또는 나열식으로 작성해주세요. 이미지에 없는 말은 적지마세요.
# 4. 테이블 풀어서 평문 또는 나열식으로 작성할 때 다른 행과 열이랑 헷갈리지 않게 값마다 잘 구분해서 적어주세요.
# 5. 테이블 해석할 때 통합셀들이 존재하니 구조를 잘 해석해서 작성해주시기 바랍니다. 어떤 게 어떤 것의 하위 내용인지를 명확히 하십시오
# 6. 당신의 의견은 궁금하지 않습니다. 해드렸습니다. 완성했습니다. 이런 표현도 적지마십시오. 이미지에 있는 내용만 적으십시오.
# 7. 만약 다단으로 구성되어져 있다면 좌측 테이블부터 먼저 작성하고 우측 테이블을 작성하십시오.
# 8. 당신에게 당신이 해석할 파일을 xml로 변경한 내용도 드리겠습니다. 페이지 해석할 때 참고하세요.
# 9. xml에 있는 텍스트는 반드시 해당 페이지에 존재한다는 겁니다. xml에 있는 텍스트를 빠트리지 마십시오.

# 자 당신이 헷갈리지 않도록 xml도 드렸습니다. 이미지를 더 잘 해석할 거라 믿습니다.
# '''


system_prompt = '''당신이 해석할 이미지는 학술 논문(Arxiv)입니다.
1. 연구 내용의 정확성이 최우선이므로 절대 요약하지 말고 문법을 지키며 보이는 그대로 작성하십시오.
2. 내용을 임의로 바꾸지 마십시오. 이미지에 보이는 모든 텍스트(본문, 각주, 캡션 등)를 다 적으십시오.
3. 테이블은 풀어서 평문 또는 나열식으로 작성하되, 행과 열의 구분이 명확하도록 값마다 잘 구분해서 적어주십시오.
4. 수식이나 방정식은 생략하지 말고 텍스트 또는 LaTeX 형식으로 정확하게 옮겨 적으십시오.
5. 테이블에 병합된 셀(Merge Cell)이 있다면 구조를 풀어서 어떤 데이터가 어떤 항목에 속하는지 명확히 하십시오.
6. 당신의 의견은 필요하지 않습니다. 인사말이나 완료 메시지 같은 불필요한 표현은 적지 말고 오직 내용만 출력하십시오.
7. 논문이 다단(Multi-column)으로 구성되어 있다면 반드시 좌측 단을 먼저 작성하고 나서 우측 단을 작성하십시오.
8. 당신에게 해당 페이지를 변환한 XML 데이터도 제공됩니다. 이미지의 텍스트가 불분명할 때 참고하십시오.
9. XML에 포함된 텍스트는 반드시 이미지 내에 존재하는 내용입니다. 이를 절대 누락하지 마십시오.

자 당신이 헷갈리지 않도록 xml도 드렸습니다. 이미지를 더 잘 해석할 거라 믿습니다.
제공된 XML을 참고하여 이미지의 학술적 내용을 정확하게 추출하십시오.'''

In [71]:
for f in all_data:
    test = f[0]
    
test

'/'

In [69]:
%%time
# 이미지와 XML 파일을 사용하여 AI 모델에 요청을 보내는 코드
result_lst = []
for file in tqdm(all_data):
    image_path = file[1]

    # 이미지를 base64 형식으로 인코딩
    base64_image = encode_image(image_path)
    with open(file[0], 'r', encoding='utf-8') as f:
        xml_content = f.read()
        
    prompt = system_prompt + xml_content + '\n시작!'
    response = client.responses.create(
    model="gpt-4.1-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": prompt},
            {
                "type": "input_image",
                "image_url": f"data:image/jpeg;base64,{base64_image}",
            },
        ],
    }],
    )
    result_lst.append(response.output_text)
    
# 결과를 페이지와 연결하여 저장
result = []
for f, r in zip(files, result_lst):
    result.append({'content': r, 'source': 'page_' + f[1].split('page_')[1]})
    
system_qna = '''당신은 주어진 2개의 연속된 논문 페이지(텍스트)로부터 연구자나 개발자가 궁금해할 만한 질문과 답변의 쌍 5개를 생성해야 합니다.

1. **문맥 파악:** 제공된 2개의 텍스트는 논문의 연속된 페이지이므로 문맥이 자연스럽게 이어집니다. 이를 고려하여 답변을 구성하십시오.
2. **데이터 구조:** 'content'는 논문의 본문 텍스트이고, 'source'는 파일명(출처)입니다.
3. **생성 목표:** 2개의 문서를 통합적으로 이해하여 심도 있는 질문(Q)과 답변(A) 5세트를 생성하십시오.
4. **형식:** Q1/A1, Q2/A2... 순서로 작성하십시오.
5. **출처 표기:** 각 답변 끝에는 반드시 참고한 문서의 'source'를 리스트 형태로 명시하십시오. (예: 출처: ['agent_2411.pdf', 'agent_2412.pdf'])
6. **Strict Content:** 주어진 텍스트('content')에 없는 외부 지식이나 환각(Hallucination)을 포함하지 마십시오. 오직 본문에 있는 내용만으로 답변해야 합니다.
7. **답변 품질:** 답변은 본문 내용을 바탕으로 최대한 상세하고 풍부하게 작성하십시오. 단답형은 지양합니다.
8. **인용 원칙(중요):** 답변 시 본문에 명시된 **섹션 번호(Section), 그림 번호(Figure), 표 번호(Table), 수식 번호(Eq)**가 있다면 이를 반드시 언급하십시오. 
   - 예: "섹션 3.2(Methodology)에 따르면...", "Table 1에 제시된 결과를 보면...", "Figure 3의 아키텍처 구조에서는..."
9. **질문 스타일:**
   - **지양(Bad):** 단순한 단어 찾기나 너무 지엽적인 수치 확인. (예: "3번째 줄의 두 번째 단어는 무엇인가요?", "알파 값이 0.001인가요?")
   - **권장(Good):** 논문의 핵심 기여, 제안하는 모델의 구조, 실험 설정, 경쟁 모델 대비 성능 차이, 한계점 등 연구자가 궁금해할 내용. (예: "제안하는 'Self-RAG' 프레임워크의 4가지 핵심 모듈은 무엇이며 각각 어떤 역할을 수행하나요?", "Table 2에서 보여주는 Llama2 모델 대비 성능 향상 폭은 어느 정도인가요?")
10. **우선순위:** 논문 내의 **불릿 포인트(List), 알고리즘 단계, 실험 결과 표, 핵심 기여(Contributions)** 내용은 매우 중요하므로 이를 바탕으로 QA를 우선 생성하십시오.

예시)
입력: 
{'content': '1. Introduction\n최근 대형언어모델(LLM)은 다양한 작업에서 뛰어난 성능을 보이지만, 여전히 환각(Hallucination) 문제와 최신 정보 부재의 한계를 가진다. 이를 해결하기 위해 검색 증강 생성(RAG)이 제안되었다.\n(중략)\n본 논문에서는 이를 개선한 Adaptive-RAG를 제안한다.\n\n2. Methodology\n2.1 Adaptive Retrieval\n제안하는 모델은 질문의 복잡도를 분류하여 검색 필요 여부를 동적으로 결정한다. Figure 1은 전체 파이프라인을 보여준다.', 'source': 'paper_part1.pdf'} 
{'content': 'Figure 1: Adaptive-RAG Pipeline. (a) Query Classifier가 난이도를 판별 (b) 필요 시 Retriever 호출 (c) Generator가 답변 생성.\n\n3. Experiments\n우리는 Natural Questions(NQ) 데이터셋을 사용하여 평가를 진행했다. Table 1은 주요 베이스라인 모델들과의 비교 결과를 보여준다. Adaptive-RAG는 기존 RAG 대비 정확도(Accuracy)가 5.4% 향상되었다.', 'source': 'paper_part2.pdf'}

답변:
Q1: 본 논문에서 기존 LLM과 RAG의 한계를 극복하기 위해 제안한 모델은 무엇이며, 어떤 핵심 아이디어를 가지고 있나요?
A1: 본 논문에서는 기존 LLM의 환각 문제와 최신 정보 부재를 해결하기 위해 **Adaptive-RAG**를 제안했습니다. 섹션 2.1(Methodology)에 따르면, 이 모델의 핵심은 질문의 복잡도를 분류하여 검색(Retrieval)이 실제로 필요한지 여부를 동적으로 결정(Adaptive Retrieval)하는 것입니다. 이를 통해 불필요한 검색 비용을 줄이고 정확도를 높이는 것을 목표로 합니다.
출처: ['paper_part1.pdf']

Q2: Figure 1에 묘사된 Adaptive-RAG의 전체 파이프라인 동작 과정은 어떻게 구성되나요?
A2: Figure 1에 따르면 Adaptive-RAG의 파이프라인은 크게 세 단계로 구성됩니다.
1. **Query Classifier:** 입력된 질문의 난이도와 복잡도를 먼저 판별합니다.
2. **Retriever 호출:** 분류 결과에 따라 검색이 필요하다고 판단될 경우에만 외부 지식베이스에서 정보를 검색합니다.
3. **Generator:** 검색된 정보(또는 검색 없이)를 바탕으로 최종 답변을 생성합니다.
이러한 구조를 통해 효율적인 리소스 활용이 가능합니다.
출처: ['paper_part1.pdf', 'paper_part2.pdf']

Q3: 본 연구의 실험 설정과 주요 결과는 어떠한가요? 특히 기존 모델 대비 성능 향상은 어느 정도입니까?
A3: 섹션 3(Experiments)에 따르면, 연구진은 **Natural Questions(NQ)** 데이터셋을 사용하여 평가를 진행했습니다. Table 1의 실험 결과를 보면, 제안하는 Adaptive-RAG 모델은 기존 RAG 모델들과 비교했을 때 **정확도(Accuracy) 측면에서 5.4%의 향상**을 기록했습니다. 이는 동적 검색 전략이 유효함을 시사합니다.
출처: ['paper_part2.pdf']

... (이하 생략)

시작!'''

my_list = result

user_prompt = []

# 리스트를 순회하면서 각 원소와 그 다음 원소만 출력
for i in range(len(my_list) - 1):  # 끝에서 두 번째 원소까지만 순회하도록 변경
    user_prompt.append('입력: ' + str(my_list[i]) + ' ' + str(my_list[i+1]) + '\n답변:')

qa_result_lst = []

previous_question = ''

with open(file_path, 'w', encoding='utf-8') as file:
    for user in tqdm(user_prompt):

        if len(previous_question) > 1:
           user = user + '\n단 아래의 질문과 답변과 유사한 내용은 피하십시오. 이미 갖고 있는 데이터입니다.\n' + previous_question + '\n위 내용과 거의 중복되고 유사한 질문 답변은 필요없습니다.'

        completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ],
        temperature=0
        )
        text = completion.choices[0].message.content
        qa_result_lst.append(text)
        json.dump(text, file, ensure_ascii=False)
        file.write('\n')
        file.flush()
        previous_question = text


  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 4.07 ms, sys: 8.23 ms, total: 12.3 ms
Wall time: 16.8 ms


FileNotFoundError: [Errno 2] No such file or directory: 'r'

In [72]:
import os
import json
import base64
from tqdm import tqdm
from openai import OpenAI

# 클라이언트 초기화
client = OpenAI()

# ----------------------------------------------------------------
# 1. 설정 및 프롬프트 정의
# ----------------------------------------------------------------

# (1) 텍스트 추출용 시스템 프롬프트 (Arxiv 논문용)
system_qna = '''당신은 주어진 2개의 연속된 논문 페이지(텍스트)로부터 연구자나 개발자가 궁금해할 만한 질문과 답변의 쌍 5개를 생성해야 합니다.

1. **문맥 파악:** 제공된 2개의 텍스트는 논문의 연속된 페이지이므로 문맥이 자연스럽게 이어집니다. 이를 고려하여 답변을 구성하십시오.
2. **데이터 구조:** 'content'는 논문의 본문 텍스트이고, 'source'는 파일명(출처)입니다.
3. **생성 목표:** 2개의 문서를 통합적으로 이해하여 심도 있는 질문(Q)과 답변(A) 5세트를 생성하십시오.
4. **형식:** Q1/A1, Q2/A2... 순서로 작성하십시오.
5. **출처 표기:** 각 답변 끝에는 반드시 참고한 문서의 'source'를 리스트 형태로 명시하십시오. (예: 출처: ['agent_2411.pdf', 'agent_2412.pdf'])
6. **Strict Content:** 주어진 텍스트('content')에 없는 외부 지식이나 환각(Hallucination)을 포함하지 마십시오. 오직 본문에 있는 내용만으로 답변해야 합니다.
7. **답변 품질:** 답변은 본문 내용을 바탕으로 최대한 상세하고 풍부하게 작성하십시오. 단답형은 지양합니다.
8. **인용 원칙(중요):** 답변 시 본문에 명시된 **섹션 번호(Section), 그림 번호(Figure), 표 번호(Table), 수식 번호(Eq)**가 있다면 이를 반드시 언급하십시오. 
   - 예: "섹션 3.2(Methodology)에 따르면...", "Table 1에 제시된 결과를 보면...", "Figure 3의 아키텍처 구조에서는..."
9. **질문 스타일:**
   - **지양(Bad):** 단순한 단어 찾기나 너무 지엽적인 수치 확인. (예: "3번째 줄의 두 번째 단어는 무엇인가요?", "알파 값이 0.001인가요?")
   - **권장(Good):** 논문의 핵심 기여, 제안하는 모델의 구조, 실험 설정, 경쟁 모델 대비 성능 차이, 한계점 등 연구자가 궁금해할 내용. (예: "제안하는 'Self-RAG' 프레임워크의 4가지 핵심 모듈은 무엇이며 각각 어떤 역할을 수행하나요?", "Table 2에서 보여주는 Llama2 모델 대비 성능 향상 폭은 어느 정도인가요?")
10. **우선순위:** 논문 내의 **불릿 포인트(List), 알고리즘 단계, 실험 결과 표, 핵심 기여(Contributions)** 내용은 매우 중요하므로 이를 바탕으로 QA를 우선 생성하십시오.

예시)
입력: 
{'content': '1. Introduction\n최근 대형언어모델(LLM)은 다양한 작업에서 뛰어난 성능을 보이지만, 여전히 환각(Hallucination) 문제와 최신 정보 부재의 한계를 가진다. 이를 해결하기 위해 검색 증강 생성(RAG)이 제안되었다.\n(중략)\n본 논문에서는 이를 개선한 Adaptive-RAG를 제안한다.\n\n2. Methodology\n2.1 Adaptive Retrieval\n제안하는 모델은 질문의 복잡도를 분류하여 검색 필요 여부를 동적으로 결정한다. Figure 1은 전체 파이프라인을 보여준다.', 'source': 'paper_part1.pdf'} 
{'content': 'Figure 1: Adaptive-RAG Pipeline. (a) Query Classifier가 난이도를 판별 (b) 필요 시 Retriever 호출 (c) Generator가 답변 생성.\n\n3. Experiments\n우리는 Natural Questions(NQ) 데이터셋을 사용하여 평가를 진행했다. Table 1은 주요 베이스라인 모델들과의 비교 결과를 보여준다. Adaptive-RAG는 기존 RAG 대비 정확도(Accuracy)가 5.4% 향상되었다.', 'source': 'paper_part2.pdf'}

답변:
Q1: 본 논문에서 기존 LLM과 RAG의 한계를 극복하기 위해 제안한 모델은 무엇이며, 어떤 핵심 아이디어를 가지고 있나요?
A1: 본 논문에서는 기존 LLM의 환각 문제와 최신 정보 부재를 해결하기 위해 **Adaptive-RAG**를 제안했습니다. 섹션 2.1(Methodology)에 따르면, 이 모델의 핵심은 질문의 복잡도를 분류하여 검색(Retrieval)이 실제로 필요한지 여부를 동적으로 결정(Adaptive Retrieval)하는 것입니다. 이를 통해 불필요한 검색 비용을 줄이고 정확도를 높이는 것을 목표로 합니다.
출처: ['paper_part1.pdf']

Q2: Figure 1에 묘사된 Adaptive-RAG의 전체 파이프라인 동작 과정은 어떻게 구성되나요?
A2: Figure 1에 따르면 Adaptive-RAG의 파이프라인은 크게 세 단계로 구성됩니다.
1. **Query Classifier:** 입력된 질문의 난이도와 복잡도를 먼저 판별합니다.
2. **Retriever 호출:** 분류 결과에 따라 검색이 필요하다고 판단될 경우에만 외부 지식베이스에서 정보를 검색합니다.
3. **Generator:** 검색된 정보(또는 검색 없이)를 바탕으로 최종 답변을 생성합니다.
이러한 구조를 통해 효율적인 리소스 활용이 가능합니다.
출처: ['paper_part1.pdf', 'paper_part2.pdf']

Q3: 본 연구의 실험 설정과 주요 결과는 어떠한가요? 특히 기존 모델 대비 성능 향상은 어느 정도입니까?
A3: 섹션 3(Experiments)에 따르면, 연구진은 **Natural Questions(NQ)** 데이터셋을 사용하여 평가를 진행했습니다. Table 1의 실험 결과를 보면, 제안하는 Adaptive-RAG 모델은 기존 RAG 모델들과 비교했을 때 **정확도(Accuracy) 측면에서 5.4%의 향상**을 기록했습니다. 이는 동적 검색 전략이 유효함을 시사합니다.
출처: ['paper_part2.pdf']

... (이하 생략)

시작!'''

# ----------------------------------------------------------------
# 2. 유틸리티 함수
# ----------------------------------------------------------------

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# ----------------------------------------------------------------
# 3. 메인 로직 실행
# ----------------------------------------------------------------

# all_data는 { '폴더경로': [('xml경로', 'png경로'), ...] } 형태라고 가정

for folder_path, files_list in all_data.items():
    print(f"\n📂 Processing Folder: {folder_path}")
    
    # 저장할 파일 경로 설정 (폴더 내부에 저장)
    extracted_json_path = os.path.join(folder_path, "extracted_texts.json")
    qa_jsonl_path = os.path.join(folder_path, "qa_dataset.jsonl")
    
    # === [Phase 1] 이미지/XML에서 텍스트 추출 ===
    extracted_results = []
    
    # 이미 추출된 파일이 있다면 로드해서 비용 절약 (선택사항)
    if os.path.exists(extracted_json_path):
        print("  - 이미 추출된 텍스트 파일이 존재하여 로드합니다.")
        with open(extracted_json_path, 'r', encoding='utf-8') as f:
            extracted_results = json.load(f)
    else:
        print("  - 텍스트 추출 시작 (GPT-4o-mini Vision)...")
        # 파일 리스트 정렬 (페이지 순서 보장)
        files_list.sort(key=lambda x: x[1]) 
        
        for xml_path, png_path in tqdm(files_list, desc="Extracting Pages"):
            if png_path == 'no png':
                continue
                
            # 이미지 인코딩
            base64_image = encode_image(png_path)
            
            # XML 읽기 (없으면 빈 문자열)
            xml_content = ""
            if xml_path != 'no xml' and os.path.exists(xml_path):
                with open(xml_path, 'r', encoding='utf-8') as f:
                    xml_content = f.read()
            
            prompt_text = system_prompt + "\n\n[XML Data]\n" + xml_content + '\n시작!'

            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini", # vision 지원 모델 (gpt-4.1-mini는 오타일 수 있어 수정함)
                    messages=[
                        {
                            "role": "user", 
                            "content": [
                                {"type": "text", "text": prompt_text},
                                {
                                    "type": "image_url",
                                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                                },
                            ],
                        }
                    ],
                    max_tokens=2000
                )
                extracted_text = response.choices[0].message.content
                
                # 결과 저장 구조
                extracted_results.append({
                    'content': extracted_text,
                    'source': os.path.basename(png_path) # 전체 경로 대신 파일명만
                })
                
            except Exception as e:
                print(f"  ❌ Error processing {png_path}: {e}")

        # 추출 결과 저장
        with open(extracted_json_path, 'w', encoding='utf-8') as f:
            json.dump(extracted_results, f, ensure_ascii=False, indent=4)
        print(f"  ✅ 텍스트 추출 완료: {len(extracted_results)} 페이지")


    # === [Phase 2] QA 데이터 생성 (Sliding Window) ===
    print("  - QA 데이터 생성 시작 (GPT-4-Turbo/Preview)...")
    
    previous_qa_context = "" # 중복 질문 방지용 히스토리
    
    # qa_dataset.jsonl 파일 열기 (append 모드 대신 write 모드로 새로 작성)
    with open(qa_jsonl_path, 'w', encoding='utf-8') as qa_file:
        
        # (i)번째 페이지와 (i+1)번째 페이지를 묶어서 처리
        # 페이지가 1개밖에 없으면 루프가 돌지 않으므로 예외처리 필요할 수 있음
        for i in tqdm(range(len(extracted_results) - 1), desc="Generating QA"):
            
            current_page = extracted_results[i]
            next_page = extracted_results[i+1]
            
            # 프롬프트 구성
            user_input_content = f"입력: {current_page} {next_page}\n답변:"
            
            # 중복 방지 로직 추가
            final_user_prompt = user_input_content
            if len(previous_qa_context) > 1:
                final_user_prompt += f"\n\n[주의] 바로 앞 페이지에서 생성된 아래 질문들과 중복되거나 매우 유사한 내용은 피하십시오:\n{previous_qa_context}"

            try:
                completion = client.chat.completions.create(
                    model="gpt-4-1106-preview", # 또는 gpt-4-turbo
                    messages=[
                        {"role": "system", "content": system_qna},
                        {"role": "user", "content": final_user_prompt}
                    ],
                    temperature=0.2 # 사실 기반이므로 온도를 낮게 설정
                )
                
                generated_qa = completion.choices[0].message.content
                
                # 결과 파일에 쓰기 (JSONL 형식: 한 줄에 하나의 JSON 문자열)
                # text 자체를 저장하거나, 구조화해서 저장할 수 있음. 여기서는 raw text 저장.
                json.dump({"qa_pair": generated_qa, "source_pages": [current_page['source'], next_page['source']]}, qa_file, ensure_ascii=False)
                qa_file.write('\n')
                qa_file.flush() # 강제 저장
                
                # 컨텍스트 업데이트 (다음 루프의 중복 방지용)
                previous_qa_context = generated_qa
                
            except Exception as e:
                print(f"  ❌ Error generating QA for {current_page['source']}: {e}")
                
    print(f"  ✅ QA 생성 완료: {qa_jsonl_path}")
    print("----------------------------------------------------------------")
    
    
    


📂 Processing Folder: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages
  - 텍스트 추출 시작 (GPT-4o-mini Vision)...


Extracting Pages: 100%|██████████| 28/28 [08:33<00:00, 18.33s/it]


  ✅ 텍스트 추출 완료: 28 페이지
  - QA 데이터 생성 시작 (GPT-4-Turbo/Preview)...


Generating QA: 100%|██████████| 27/27 [19:07<00:00, 42.50s/it]


  ✅ QA 생성 완료: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2505.30656746v1.full_pages/qa_dataset.jsonl
----------------------------------------------------------------

📂 Processing Folder: /Users/seobi/PythonProjects/AxDeepScholar/src/data/agent/agent_2411.00114v1_pages
  - 텍스트 추출 시작 (GPT-4o-mini Vision)...


...

  ❌ Error processing /Users/seobi/PythonProjects/AxDeepScholar/src/data/rag/rag_2409.09916v1_pages/page_8.png: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


Extracting Pages: 100%|██████████| 12/12 [01:17<00:00,  6.44s/it]


  ❌ Error processing /Users/seobi/PythonProjects/AxDeepScholar/src/data/rag/rag_2409.09916v1_pages/page_9.png: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
  ✅ 텍스트 추출 완료: 0 페이지
  - QA 데이터 생성 시작 (GPT-4-Turbo/Preview)...


Generating QA: 0it [00:00, ?it/s]

  ✅ QA 생성 완료: /Users/seobi/PythonProjects/AxDeepScholar/src/data/rag/rag_2409.09916v1_pages/qa_dataset.jsonl
----------------------------------------------------------------
